Need to compare 2 different algorithms for selecting a driver for a booking. 2 different methods were used, labelled A and B. Comparison table is shown at the bottom of this notebook, along with a summary of the findings.
post Copy 1: remove paths to files, run everything from one folder, will need to run this after cd to the directory with the files. 
cd ..\OnlineTaxi
jupyter notebook

### step 1: load the data files into pandas dataframes

In [1]:
import pandas as pd
import datetime as dt

df_booking = pd.read_csv("booking_log.csv")
df_experiment = pd.read_csv("experiment_log.csv")
df_participant = pd.read_csv("participant_log.csv")

### **step 2: clean up the data**

#### I am look at the booking table first

|                  event_timestamp|      order_id| booking_status|   customer_id|
|:--------------------------------|:-------------|:-------------:|:-------------|
|0     2015-05-12 05:25:23.904 UTC|  102254181112|        CREATED|  715281995014|   
|1      2015-05-12 05:25:24.67 UTC|  102254181112|   DRIVER_FOUND|  715281995014|   
|2  2015-05-12 05:55:10.496544 UTC|  102254181112|      COMPLETED|  715281995014|  
|3     2015-05-12 05:29:37.305 UTC|  105520285203|        CREATED|  957980952237|   
|4     2015-05-12 05:29:38.214 UTC|  105520285203|   DRIVER_FOUND|  957980952237|

* <font color='green'>look for <u>null</u> values</font>

In [2]:
null_columns=df_booking.columns[df_booking.isnull().any()]
df_booking[null_columns].isnull().sum()

event_timestamp        92
driver_id          194035
dtype: int64

I found a few columns with missing data, check to see if this causes issues with data type autodetection
* look at data types



In [3]:
print(df_booking.head(5))
print("")
print(df_booking.dtypes)

                  event_timestamp      order_id booking_status   customer_id  \
0     2015-05-12 05:25:23.904 UTC  102254181112        CREATED  715281995014   
1      2015-05-12 05:25:24.67 UTC  102254181112   DRIVER_FOUND  715281995014   
2  2015-05-12 05:55:10.496544 UTC  102254181112      COMPLETED  715281995014   
3     2015-05-12 05:29:37.305 UTC  105520285203        CREATED  957980952237   
4     2015-05-12 05:29:38.214 UTC  105520285203   DRIVER_FOUND  957980952237   

      driver_id  trip_distance  pickup_latitude  pickup_longitude  
0           NaN            2.0        -2.998461        104.788077  
1  2.211727e+11            2.0        -2.998461        104.788077  
2  2.211727e+11            2.0        -2.998461        104.788077  
3           NaN            2.0        -2.970621        104.756226  
4  5.276724e+11            2.0        -2.970621        104.756226  

event_timestamp      object
order_id              int64
booking_status       object
customer_id           int6

find the rows with null in the timestamp column

In [4]:
print(df_booking[df_booking["event_timestamp"].isnull()][null_columns])

       event_timestamp     driver_id
1345               NaN  2.712591e+11
8780               NaN  7.330471e+11
13993              NaN  6.372264e+11
29544              NaN  4.114654e+11
58836              NaN  9.250986e+11
...                ...           ...
657580             NaN  1.814914e+11
660349             NaN  9.674390e+11
668346             NaN  8.106505e+11
670737             NaN  2.993758e+11
691028             NaN  1.630862e+11

[92 rows x 2 columns]


remove these rows so we can convert the timestamp column to a datetime datatype (get an error otherwise)
and then check if the nulls were removed. We can ignore these rows as there only a small percentage and the missing data makes the entire row suspect.

In [5]:
df_booking_clean = df_booking.dropna(subset=['event_timestamp'])
print(df_booking_clean[df_booking_clean["event_timestamp"].isnull()][null_columns])

Empty DataFrame
Columns: [event_timestamp, driver_id]
Index: []


success, no more nulls!
now see if we can convert the timestamp column to a datetime datatype
we remove the milliseconds and UTC from the data as they are not needed and are missing occasionally, causing issues with the strptime format

In [6]:
df_booking_clean.head(5)

,event_timestamp,order_id,booking_status,customer_id,driver_id,trip_distance,pickup_latitude,pickup_longitude
0,2015-05-12 05:25:23.904 UTC,102254181112,CREATED,715281995014,NaN,2.0,-2.998461,104.788077
1,2015-05-12 05:25:24.67 UTC,102254181112,DRIVER_FOUND,715281995014,2.211727e+11,2.0,-2.998461,104.788077
2,2015-05-12 05:55:10.496544 UTC,102254181112,COMPLETED,715281995014,2.211727e+11,2.0,-2.998461,104.788077
3,2015-05-12 05:29:37.305 UTC,105520285203,CREATED,957980952237,NaN,2.0,-2.970621,104.756226
4,2015-05-12 05:29:38.214 UTC,105520285203,DRIVER_FOUND,957980952237,5.276724e+11,2.0,-2.970621,104.756226


In [7]:
df_booking_clean['event_timestamp'] = df_booking_clean['event_timestamp'].str.slice(start=0, stop=19, step=1).apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
print(df_booking_clean.dtypes)

event_timestamp     datetime64[ns]
order_id                     int64
booking_status              object
customer_id                  int64
driver_id                  float64
trip_distance              float64
pickup_latitude            float64
pickup_longitude           float64
dtype: object


c:\users\iowahawk89\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


now we can look at the empty driver_id columns. This is a larger percentage, but we may require the driver_id value for the ML algorithm later. But first need to see why so many are missing.

In [8]:
print(df_booking_clean[df_booking_clean["driver_id"].isnull()])

           event_timestamp      order_id booking_status   customer_id  \
0      2015-05-12 05:25:23  102254181112        CREATED  715281995014   
3      2015-05-12 05:29:37  105520285203        CREATED  957980952237   
6      2015-05-18 04:32:01  113947347802        CREATED  352787792790   
10     2015-05-13 04:10:24  197309126170        CREATED  624841359255   
13     2015-05-15 09:11:43  209915120110        CREATED  725333969587   
...                    ...           ...            ...           ...   
699196 2015-05-18 13:35:05  513429812000        CREATED  809741484328   
699198 2015-05-11 06:32:33  883402014715        CREATED  535664745680   
699202 2015-05-12 03:19:24  929201302901        CREATED  215391415102   
699205 2015-05-18 10:21:10  995905396208        CREATED  366602285072   
699208 2015-05-18 15:04:32  997608900030        CREATED  587801587010   

        driver_id  trip_distance  pickup_latitude  pickup_longitude  
0             NaN          2.000        -2.998461    

no driver when an order is created, driver is added later... so we will keep these records.

In [9]:
null_columns=df_booking_clean.columns[df_booking_clean.isnull().any()]
df_booking_clean[null_columns].isnull().sum()

driver_id    194035
dtype: int64

next we need to modify the driver_id dtype from float to int64 to match order_id and customer_id

set driver_id to 0 for null values to allow conversion

In [10]:
values = {'driver_id': 0}
df_booking_clean2 = df_booking_clean.fillna(value=values)
df_booking_clean2

,event_timestamp,order_id,booking_status,customer_id,driver_id,trip_distance,pickup_latitude,pickup_longitude
0,2015-05-12 05:25:23,102254181112,CREATED,715281995014,0.000000e+00,2.000,-2.998461,104.788077
1,2015-05-12 05:25:24,102254181112,DRIVER_FOUND,715281995014,2.211727e+11,2.000,-2.998461,104.788077
2,2015-05-12 05:55:10,102254181112,COMPLETED,715281995014,2.211727e+11,2.000,-2.998461,104.788077
3,2015-05-12 05:29:37,105520285203,CREATED,957980952237,0.000000e+00,2.000,-2.970621,104.756226
4,2015-05-12 05:29:38,105520285203,DRIVER_FOUND,957980952237,5.276724e+11,2.000,-2.970621,104.756226
...,...,...,...,...,...,...,...,...
699207,2015-05-18 10:22:01,995905396208,CUSTOMER_CANCELLED,366602285072,6.149728e+11,9.916,-2.986197,104.761670
699208,2015-05-18 15:04:32,997608900030,CREATED,587801587010,0.000000e+00,9.916,-2.979303,104.745228
699209,2015-05-18 15:04:33,997608900030,DRIVER_FOUND,587801587010,4.395662e+11,9.916,-2.979303,104.745228
699210,2015-05-18 15:10:44,997608900030,PICKED_UP,587801587010,4.395662e+11,9.916,-2.979303,104.745228


now check if there are any nulls left

In [11]:
print(df_booking_clean2[df_booking_clean2["driver_id"].isnull()])

Empty DataFrame
Columns: [event_timestamp, order_id, booking_status, customer_id, driver_id, trip_distance, pickup_latitude, pickup_longitude]
Index: []


convert driver_id to Int now

In [12]:
df_booking_clean2['driver_id'] = df_booking_clean2['driver_id'].astype('int64')
print(df_booking_clean2.dtypes)

event_timestamp     datetime64[ns]
order_id                     int64
booking_status              object
customer_id                  int64
driver_id                    int64
trip_distance              float64
pickup_latitude            float64
pickup_longitude           float64
dtype: object


In [13]:
print(df_experiment.head(5))
print("")
print(df_experiment.dtypes)

               event_timestamp  experiment_key      order_id   customer_id  \
0  2015-05-10 02:18:28.641 UTC    324759489928  244229443402  100009689647   
1  2015-05-18 13:24:20.642 UTC    603052383530  696348384208  100023146606   
2  2015-05-11 09:08:49.766 UTC    522992651818  694879938150  100179926967   
3  2015-05-10 03:04:10.056 UTC    130535259984  991024621819  100203429317   
4  2015-05-16 08:50:12.194 UTC    695675818430  379357397810  100321038124   

  experiment_tag  
0              A  
1              A  
2              A  
3              A  
4              A  

event_timestamp    object
experiment_key      int64
order_id            int64
customer_id         int64
experiment_tag     object
dtype: object


In [14]:
null_columns=df_experiment.columns[df_experiment.isnull().any()]
df_experiment[null_columns].isnull().sum()
df_experiment['event_timestamp'] = df_experiment['event_timestamp'].str.slice(start=0, stop=19, step=1).apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

In [15]:
print(df_experiment.head(5))
print("")
print(df_experiment.dtypes)

      event_timestamp  experiment_key      order_id   customer_id  \
0 2015-05-10 02:18:28    324759489928  244229443402  100009689647   
1 2015-05-18 13:24:20    603052383530  696348384208  100023146606   
2 2015-05-11 09:08:49    522992651818  694879938150  100179926967   
3 2015-05-10 03:04:10    130535259984  991024621819  100203429317   
4 2015-05-16 08:50:12    695675818430  379357397810  100321038124   

  experiment_tag  
0              A  
1              A  
2              A  
3              A  
4              A  

event_timestamp    datetime64[ns]
experiment_key              int64
order_id                    int64
customer_id                 int64
experiment_tag             object
dtype: object


In [16]:
print(df_participant.head(5))
print("")
print(df_participant.dtypes)

               event_timestamp     driver_id participant_status      order_id  \
0  2015-05-18 11:45:19.519 UTC  110283102582            CREATED  938125906616   
1    2015-05-18 11:45:29.8 UTC  110283102582           ACCEPTED  938125906616   
2  2015-05-11 10:35:57.021 UTC  126346998414            CREATED  992080359113   
3  2015-05-11 10:36:01.148 UTC  126346998414           ACCEPTED  992080359113   
4  2015-05-16 12:34:20.044 UTC  199325048812           ACCEPTED  797723880713   

   experiment_key  driver_latitude  driver_longitude  driver_gps_accuracy  
0    309579604593        -2.981389        104.761898                  2.0  
1    309579604593        -2.981389        104.761898                  2.0  
2    941995970460        -2.966915        104.748040                  2.0  
3    941995970460        -2.966915        104.748040                  2.0  
4    197604093364        -3.014740        104.694647                  2.0  

event_timestamp         object
driver_id                

In [17]:
null_columns=df_participant.columns[df_participant.isnull().any()]
df_participant[null_columns].isnull().sum()
df_participant['event_timestamp'] = df_participant['event_timestamp'].str.slice(start=0, stop=19, step=1).apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

In [18]:
print(df_participant.head(5))
print("")
print(df_participant.dtypes)

      event_timestamp     driver_id participant_status      order_id  \
0 2015-05-18 11:45:19  110283102582            CREATED  938125906616   
1 2015-05-18 11:45:29  110283102582           ACCEPTED  938125906616   
2 2015-05-11 10:35:57  126346998414            CREATED  992080359113   
3 2015-05-11 10:36:01  126346998414           ACCEPTED  992080359113   
4 2015-05-16 12:34:20  199325048812           ACCEPTED  797723880713   

   experiment_key  driver_latitude  driver_longitude  driver_gps_accuracy  
0    309579604593        -2.981389        104.761898                  2.0  
1    309579604593        -2.981389        104.761898                  2.0  
2    941995970460        -2.966915        104.748040                  2.0  
3    941995970460        -2.966915        104.748040                  2.0  
4    197604093364        -3.014740        104.694647                  2.0  

event_timestamp        datetime64[ns]
driver_id                       int64
participant_status             obj

To calculate booking conversion rate by experiment_tag need to merge the corresponding tables together and calculate if the booking status flow includes CREATED and COMPLETED or not.

In [19]:
df_booking_clean2.head(5)

,event_timestamp,order_id,booking_status,customer_id,driver_id,trip_distance,pickup_latitude,pickup_longitude
0,2015-05-12 05:25:23,102254181112,CREATED,715281995014,0,2.0,-2.998461,104.788077
1,2015-05-12 05:25:24,102254181112,DRIVER_FOUND,715281995014,221172732473,2.0,-2.998461,104.788077
2,2015-05-12 05:55:10,102254181112,COMPLETED,715281995014,221172732473,2.0,-2.998461,104.788077
3,2015-05-12 05:29:37,105520285203,CREATED,957980952237,0,2.0,-2.970621,104.756226
4,2015-05-12 05:29:38,105520285203,DRIVER_FOUND,957980952237,527672421312,2.0,-2.970621,104.756226


In [20]:
df_booking_clean2.booking_status.unique()

array(['CREATED', 'DRIVER_FOUND', 'COMPLETED', 'CUSTOMER_CANCELLED',
       'PICKED_UP', 'DRIVER_NOT_FOUND', 'DRIVER_CANCELLED'], dtype=object)

find order(s) with most rows to use as example

In [21]:
df_booking_clean2.groupby(['order_id']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[False])

,order_id,size
174714,946291474301,11
2380,111487998703,10
171050,928523273405,10
68237,431319739619,9
23491,215093953304,9
...,...,...
2196,110678736904,1
128440,722441728313,1
165122,899594123704,1
29718,244998578719,1


does a customer_id need to be included as the index?

In [22]:
df_booking_clean2.groupby(['order_id','customer_id']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[False])

,order_id,customer_id,size
174714,946291474301,853916781308,11
2380,111487998703,206906066358,10
171050,928523273405,734252822849,10
68237,431319739619,801689095557,9
23491,215093953304,953993588212,9
...,...,...,...
2196,110678736904,556347535574,1
128440,722441728313,650175876977,1
165122,899594123704,983186951957,1
29718,244998578719,340425733378,1


In [23]:
df_booking_clean2.groupby(['customer_id']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[False])

,customer_id,size
31076,375872381480,83
34922,410231096437,83
21499,290683768488,81
38418,439705670853,70
54837,584401166017,69
...,...,...
11223,199808089813,1
50622,546630978972,1
98731,972537368347,1
92352,916414706624,1


In [24]:
df_booking_clean2.query('order_id == 215093953304')

,event_timestamp,order_id,booking_status,customer_id,driver_id,trip_distance,pickup_latitude,pickup_longitude
475450,2015-05-18 14:29:53,215093953304,CREATED,953993588212,0,2.239,-2.974416,104.742138
475451,2015-05-18 14:31:46,215093953304,PICKED_UP,953993588212,517406231140,2.239,-2.974416,104.742138
475452,2015-05-18 14:31:46,215093953304,PICKED_UP,953993588212,517406231140,2.239,-2.974416,104.742138
475453,2015-05-18 14:31:46,215093953304,PICKED_UP,953993588212,517406231140,2.239,-2.974416,104.742138
475454,2015-05-18 14:29:54,215093953304,DRIVER_FOUND,953993588212,517406231140,2.239,-2.974416,104.742138
475455,2015-05-18 14:31:46,215093953304,PICKED_UP,953993588212,517406231140,2.239,-2.974416,104.742138
475456,2015-05-18 14:31:46,215093953304,PICKED_UP,953993588212,517406231140,2.239,-2.974416,104.742138
475457,2015-05-18 14:31:46,215093953304,PICKED_UP,953993588212,517406231140,2.239,-2.974416,104.742138
475458,2015-05-18 14:36:21,215093953304,COMPLETED,953993588212,517406231140,2.239,-2.974416,104.742138


table needs further cleaning - looks like a lot of duplicates here under the PICKED_UP status, should remove duplicates as they will skew any reporting or ML algorithms. 

In [25]:
df_booking_clean2.drop_duplicates(subset=None, keep='first', inplace=True)
df_booking_clean2.query('order_id == 215093953304')

,event_timestamp,order_id,booking_status,customer_id,driver_id,trip_distance,pickup_latitude,pickup_longitude
475450,2015-05-18 14:29:53,215093953304,CREATED,953993588212,0,2.239,-2.974416,104.742138
475451,2015-05-18 14:31:46,215093953304,PICKED_UP,953993588212,517406231140,2.239,-2.974416,104.742138
475454,2015-05-18 14:29:54,215093953304,DRIVER_FOUND,953993588212,517406231140,2.239,-2.974416,104.742138
475458,2015-05-18 14:36:21,215093953304,COMPLETED,953993588212,517406231140,2.239,-2.974416,104.742138


look at data for a single customer

In [26]:
df_booking_clean2.query('customer_id == 375872381480').sort_values(['event_timestamp'], ascending=[True])

,event_timestamp,order_id,booking_status,customer_id,driver_id,trip_distance,pickup_latitude,pickup_longitude
576641,2015-05-10 00:13:04,889149827613,CREATED,375872381480,0,3.541,-2.978718,104.775194
576642,2015-05-10 00:13:06,889149827613,DRIVER_FOUND,375872381480,645609972910,3.541,-2.978718,104.775194
576643,2015-05-10 00:21:11,889149827613,PICKED_UP,375872381480,645609972910,3.541,-2.978718,104.775194
576644,2015-05-10 00:21:45,889149827613,CUSTOMER_CANCELLED,375872381480,645609972910,3.541,-2.978718,104.775194
280929,2015-05-10 03:17:58,180518537114,CREATED,375872381480,0,3.488,-2.981180,104.752377
...,...,...,...,...,...,...,...,...
272794,2015-05-17 23:56:30,837102272907,CUSTOMER_CANCELLED,375872381480,764469437992,1.348,-2.978744,104.775167
304870,2015-05-18 08:04:59,411280499908,CREATED,375872381480,0,1.112,-2.978751,104.775163
304871,2015-05-18 08:05:01,411280499908,DRIVER_FOUND,375872381480,871115263236,1.112,-2.978751,104.775163
304872,2015-05-18 08:06:49,411280499908,PICKED_UP,375872381480,871115263236,1.112,-2.978751,104.775163


see how many rows are still here

In [27]:
print(len(df_booking))
print(len(df_booking_clean2))

699212
698141


In [28]:
df_experiment.head(5)

,event_timestamp,experiment_key,order_id,customer_id,experiment_tag
0,2015-05-10 02:18:28,324759489928,244229443402,100009689647,A
1,2015-05-18 13:24:20,603052383530,696348384208,100023146606,A
2,2015-05-11 09:08:49,522992651818,694879938150,100179926967,A
3,2015-05-10 03:04:10,130535259984,991024621819,100203429317,A
4,2015-05-16 08:50:12,695675818430,379357397810,100321038124,A


Now look for duplicates in the df_experiment dataframe

In [29]:
df_experiment.groupby(['order_id']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[False])

,order_id,size
129637,729223336804,4
97535,573536979314,4
73811,459017512914,4
81584,497237139000,4
5376,125617654309,4
...,...,...
61865,400356198303,1
61866,400358058909,1
61867,400374862711,1
61868,400376251700,1


In [30]:
df_experiment.query('order_id == 729223336804')

,event_timestamp,experiment_key,order_id,customer_id,experiment_tag
137566,2015-05-17 02:34:56,269466100851,729223336804,740387485030,B
137567,2015-05-17 02:34:56,269466100851,729223336804,740387485030,B
137569,2015-05-17 02:34:56,269466100851,729223336804,740387485030,B
137570,2015-05-17 02:34:56,269466100851,729223336804,740387485030,B


In [31]:
df_experiment.drop_duplicates(subset=None, keep='first', inplace=True)
df_experiment.query('order_id == 729223336804')

,event_timestamp,experiment_key,order_id,customer_id,experiment_tag
137566,2015-05-17 02:34:56,269466100851,729223336804,740387485030,B


In [32]:
print(len(df_experiment))

185507


In [33]:
df_participant.head(5)

,event_timestamp,driver_id,participant_status,order_id,experiment_key,driver_latitude,driver_longitude,driver_gps_accuracy
0,2015-05-18 11:45:19,110283102582,CREATED,938125906616,309579604593,-2.981389,104.761898,2.0
1,2015-05-18 11:45:29,110283102582,ACCEPTED,938125906616,309579604593,-2.981389,104.761898,2.0
2,2015-05-11 10:35:57,126346998414,CREATED,992080359113,941995970460,-2.966915,104.748040,2.0
3,2015-05-11 10:36:01,126346998414,ACCEPTED,992080359113,941995970460,-2.966915,104.748040,2.0
4,2015-05-16 12:34:20,199325048812,ACCEPTED,797723880713,197604093364,-3.014740,104.694647,2.0


In [34]:
df_participant.participant_status.unique()

array(['CREATED', 'ACCEPTED', 'IGNORED', 'REJECTED'], dtype=object)

In [35]:
df_participant.groupby(['order_id']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[False])

,order_id,size
3977,119369001704,26
163347,917815470413,20
4303,121051078603,18
25776,230075624911,18
68016,441391187120,16
...,...,...
40513,303702738807,1
92541,564317342105,1
165448,928420483107,1
796,104053517170,1


In [36]:
df_participant.query('order_id == 119369001704').sort_values('event_timestamp', ascending=True)

,event_timestamp,driver_id,participant_status,order_id,experiment_key,driver_latitude,driver_longitude,driver_gps_accuracy
132981,2015-05-11 10:16:50,347825505488,CREATED,119369001704,818360755945,-2.984148,104.761696,16.080
311617,2015-05-11 10:17:05,390184160589,CREATED,119369001704,818360755945,-2.982071,104.764466,50.664
132980,2015-05-11 10:17:05,347825505488,IGNORED,119369001704,818360755945,-2.984148,104.761696,16.080
311616,2015-05-11 10:17:14,390184160589,ACCEPTED,119369001704,818360755945,-2.982071,104.764466,50.664
227285,2015-05-11 10:41:16,606420884891,CREATED,119369001704,211400956109,-2.981984,104.765648,16.101
227286,2015-05-11 10:41:31,606420884891,IGNORED,119369001704,211400956109,-2.981984,104.765648,16.101
145965,2015-05-11 10:41:32,238871943926,CREATED,119369001704,211400956109,-2.975520,104.770673,29.669
145964,2015-05-11 10:41:41,238871943926,ACCEPTED,119369001704,211400956109,-2.975520,104.770673,29.669
252934,2015-05-11 10:52:16,671863651933,CREATED,119369001704,709862526464,-2.975327,104.768599,12.499
252933,2015-05-11 10:52:29,671863651933,ACCEPTED,119369001704,709862526464,-2.975327,104.768599,12.499


found more duplicates, this time in the participant df

In [37]:
df_participant.drop_duplicates(subset=None, keep='first', inplace=True)
df_participant.query('order_id == 119369001704')

,event_timestamp,driver_id,participant_status,order_id,experiment_key,driver_latitude,driver_longitude,driver_gps_accuracy
83908,2015-05-11 11:13:29,127836492738,IGNORED,119369001704,390407700376,-2.980679,104.770810,12.500
83909,2015-05-11 11:13:14,127836492738,CREATED,119369001704,390407700376,-2.980679,104.770810,12.500
132980,2015-05-11 10:17:05,347825505488,IGNORED,119369001704,818360755945,-2.984148,104.761696,16.080
132981,2015-05-11 10:16:50,347825505488,CREATED,119369001704,818360755945,-2.984148,104.761696,16.080
145964,2015-05-11 10:41:41,238871943926,ACCEPTED,119369001704,211400956109,-2.975520,104.770673,29.669
145965,2015-05-11 10:41:32,238871943926,CREATED,119369001704,211400956109,-2.975520,104.770673,29.669
155433,2015-05-11 11:05:22,433456969911,CREATED,119369001704,426009935277,-2.984883,104.760400,25.642
155434,2015-05-11 11:05:23,433456969911,ACCEPTED,119369001704,426009935277,-2.984883,104.760400,25.642
213522,2015-05-11 11:12:43,224241099640,IGNORED,119369001704,390407700376,-2.979211,104.769948,9.102
213523,2015-05-11 11:12:28,224241099640,CREATED,119369001704,390407700376,-2.979211,104.769948,9.102


In [38]:
print(len(df_participant))

400104


In [39]:
df_booking_clean2.query('order_id == 119369001704').sort_values('event_timestamp', ascending=True)

,event_timestamp,order_id,booking_status,customer_id,driver_id,trip_distance,pickup_latitude,pickup_longitude
679762,2015-05-11 10:16:50,119369001704,CREATED,223476004540,0,11.848,-2.981028,104.767075
679763,2015-05-11 10:17:14,119369001704,DRIVER_FOUND,223476004540,390184160589,11.848,-2.981028,104.767075


In [40]:
df_experiment.groupby(['order_id']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[False])

,order_id,size
0,100004630900,1
123664,700381094080,1
123666,700394333419,1
123667,700397691603,1
123668,700399502008,1
...,...,...
61837,400183016113,1
61838,400183611009,1
61839,400184109015,1
61840,400184427307,1


In [41]:
df_experiment.groupby(['experiment_tag']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[False])

,experiment_tag,size
1,B,166912
0,A,18595


merge the 2 dataframes into one

In [42]:
df_summary = df_booking_clean2.merge(df_experiment, left_on='order_id', right_on='order_id')

In [43]:
df_summary.head(5)

,event_timestamp_x,order_id,booking_status,customer_id_x,driver_id,trip_distance,pickup_latitude,pickup_longitude,event_timestamp_y,experiment_key,customer_id_y,experiment_tag
0,2015-05-12 05:25:23,102254181112,CREATED,715281995014,0,2.0,-2.998461,104.788077,2015-05-12 05:25:23,697155500625,715281995014,B
1,2015-05-12 05:25:24,102254181112,DRIVER_FOUND,715281995014,221172732473,2.0,-2.998461,104.788077,2015-05-12 05:25:23,697155500625,715281995014,B
2,2015-05-12 05:55:10,102254181112,COMPLETED,715281995014,221172732473,2.0,-2.998461,104.788077,2015-05-12 05:25:23,697155500625,715281995014,B
3,2015-05-12 05:29:37,105520285203,CREATED,957980952237,0,2.0,-2.970621,104.756226,2015-05-12 05:29:37,243283239235,957980952237,A
4,2015-05-12 05:29:38,105520285203,DRIVER_FOUND,957980952237,527672421312,2.0,-2.970621,104.756226,2015-05-12 05:29:37,243283239235,957980952237,A


this is the total # of orders created per tag

In [44]:
a_cr = df_summary.query('booking_status == "CREATED"').groupby(['experiment_tag']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[True])

this is the number of orders completed per tag

In [45]:
b_cr = df_summary.query('booking_status == "COMPLETED"').groupby(['experiment_tag']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[True])

In [46]:
print("created")
a_cr

created


,experiment_tag,size
0,A,18595
1,B,166911


In [47]:
print("completed")
b_cr

completed


,experiment_tag,size
0,A,13924
1,B,138107


In [48]:
print("results for A") 
BCR_A = round(b_cr.iloc[0]['size']/a_cr.iloc[0]['size'], 2)
BCR_A

results for A


0.75

In [49]:
print("results for B") 
BCR_B = round(b_cr.iloc[1]['size']/a_cr.iloc[1]['size'], 2)
BCR_B

results for B


0.83

In [50]:
df_summary.head(5)

,event_timestamp_x,order_id,booking_status,customer_id_x,driver_id,trip_distance,pickup_latitude,pickup_longitude,event_timestamp_y,experiment_key,customer_id_y,experiment_tag
0,2015-05-12 05:25:23,102254181112,CREATED,715281995014,0,2.0,-2.998461,104.788077,2015-05-12 05:25:23,697155500625,715281995014,B
1,2015-05-12 05:25:24,102254181112,DRIVER_FOUND,715281995014,221172732473,2.0,-2.998461,104.788077,2015-05-12 05:25:23,697155500625,715281995014,B
2,2015-05-12 05:55:10,102254181112,COMPLETED,715281995014,221172732473,2.0,-2.998461,104.788077,2015-05-12 05:25:23,697155500625,715281995014,B
3,2015-05-12 05:29:37,105520285203,CREATED,957980952237,0,2.0,-2.970621,104.756226,2015-05-12 05:29:37,243283239235,957980952237,A
4,2015-05-12 05:29:38,105520285203,DRIVER_FOUND,957980952237,527672421312,2.0,-2.970621,104.756226,2015-05-12 05:29:37,243283239235,957980952237,A


In [51]:
df_summary.drop_duplicates(subset=None, keep='first', inplace=True)

look for duplicates before we can pivot the table

In [52]:
df_summary.groupby(['order_id','booking_status']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[False])

,order_id,booking_status,size
227138,393177882003,COMPLETED,2
0,100004630900,COMPLETED,1
465160,700596771414,DRIVER_FOUND,1
465153,700596297701,DRIVER_FOUND,1
465154,700596677008,COMPLETED,1
...,...,...,...
232581,400156849519,COMPLETED,1
232582,400156849519,CREATED,1
232583,400156849519,DRIVER_FOUND,1
232584,400156849519,PICKED_UP,1


In [53]:
df_summary.query('order_id == 393177882003') 

,event_timestamp_x,order_id,booking_status,customer_id_x,driver_id,trip_distance,pickup_latitude,pickup_longitude,event_timestamp_y,experiment_key,customer_id_y,experiment_tag
406807,2015-05-16 09:07:05,393177882003,CREATED,919442848300,0,2.447,-2.986258,104.761606,2015-05-16 09:07:06,846647535993,919442848300,B
406808,2015-05-16 09:09:55,393177882003,PICKED_UP,919442848300,326841064192,2.447,-2.986258,104.761606,2015-05-16 09:07:06,846647535993,919442848300,B
406809,2015-05-16 09:17:58,393177882003,COMPLETED,919442848300,326841064192,2.447,-2.986258,104.761606,2015-05-16 09:07:06,846647535993,919442848300,B
406810,2015-05-16 09:07:07,393177882003,DRIVER_FOUND,919442848300,326841064192,2.447,-2.986258,104.761606,2015-05-16 09:07:06,846647535993,919442848300,B
406811,2015-05-16 09:17:59,393177882003,COMPLETED,919442848300,326841064192,2.447,-2.986258,104.761606,2015-05-16 09:07:06,846647535993,919442848300,B


still have some duplicate data so need to use pivot_table and an aggregate function to handle the duplicate index

In [54]:
df_summary_pivot = df_summary.pivot_table(index=['order_id','experiment_tag'], columns='booking_status', values='event_timestamp_x', aggfunc='min')

In [55]:
df_summary_pivot

,booking_status,COMPLETED,CREATED,CUSTOMER_CANCELLED,DRIVER_CANCELLED,DRIVER_FOUND,DRIVER_NOT_FOUND,PICKED_UP
order_id,experiment_tag,,,,,,,
100004630900,A,2015-05-12 00:27:09,2015-05-12 00:21:29,NaT,NaT,2015-05-12 00:21:30,NaT,2015-05-12 00:23:40
100005695817,A,2015-05-16 02:21:07,2015-05-16 02:09:16,NaT,NaT,2015-05-16 02:09:17,NaT,2015-05-16 02:12:06
100021292050,B,2015-05-10 13:34:21,2015-05-10 13:26:24,NaT,NaT,2015-05-10 13:26:26,NaT,2015-05-10 13:28:24
100022287505,B,2015-05-12 10:07:06,2015-05-12 09:50:51,NaT,NaT,2015-05-12 09:50:56,NaT,2015-05-12 09:54:29
100024200607,B,2015-05-11 04:57:03,2015-05-11 04:50:26,NaT,NaT,2015-05-11 04:50:27,NaT,2015-05-11 04:53:18
...,...,...,...,...,...,...,...,...
999979560817,A,2015-05-16 07:01:05,2015-05-16 06:46:51,NaT,NaT,2015-05-16 06:46:52,NaT,2015-05-16 06:50:04
999983585012,B,2015-05-18 05:52:47,2015-05-18 05:35:28,NaT,NaT,2015-05-18 05:35:30,NaT,2015-05-18 05:38:19
999990809802,B,2015-05-17 16:40:02,2015-05-17 16:28:11,NaT,NaT,2015-05-17 16:28:18,NaT,2015-05-17 16:36:30


feature engineering: add column with calculated picked_up time

In [56]:
df_summary_pivot['pick_up_time'] = df_summary_pivot.apply(lambda row: row.PICKED_UP - row.CREATED, axis=1)

In [57]:
df_summary_pivot.head(2)

,booking_status,COMPLETED,CREATED,CUSTOMER_CANCELLED,DRIVER_CANCELLED,DRIVER_FOUND,DRIVER_NOT_FOUND,PICKED_UP,pick_up_time
order_id,experiment_tag,,,,,,,,
100004630900,A,2015-05-12 00:27:09,2015-05-12 00:21:29,NaT,NaT,2015-05-12 00:21:30,NaT,2015-05-12 00:23:40,00:02:11
100005695817,A,2015-05-16 02:21:07,2015-05-16 02:09:16,NaT,NaT,2015-05-16 02:09:17,NaT,2015-05-16 02:12:06,00:02:50


perform metric calculation, comparing pick up time between experiment labels

In [58]:
a_pu = df_summary_pivot.query('experiment_tag == "A"').agg({'pick_up_time': ['mean']})
a_pu

,pick_up_time
mean,00:04:45.771325


In [59]:
b_pu = df_summary_pivot.query('experiment_tag == "B"').agg({'pick_up_time': ['mean']})
b_pu

,pick_up_time
mean,00:04:48.462690


In [60]:
print("results for A") 
MPUT_A = round( a_pu.iloc[0]['pick_up_time'].total_seconds() / 60, 2)
MPUT_A

results for A


4.76

In [61]:
print("results for B") 
MPUT_B = round( b_pu.iloc[0]['pick_up_time'].total_seconds() / 60, 2)
MPUT_B

results for B


4.81

other metrics, 1st is ratio of participant_status that is ignored or rejected to all values, 2nd is total_time of order (created to completed)

In [62]:
df_participant.groupby(['order_id','participant_status']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[False])

,order_id,participant_status,size
8303,119369001704,CREATED,13
339727,917815470413,CREATED,10
8978,121051078603,CREATED,9
53653,230075624911,CREATED,9
172402,516674224907,CREATED,8
...,...,...,...
128415,409336820605,ACCEPTED,1
128414,409336802080,IGNORED,1
128412,409336802080,ACCEPTED,1
128411,409333031807,CREATED,1


In [63]:
df_participant.query('order_id == 121051078603').sort_values(['event_timestamp'], ascending=[True])

,event_timestamp,driver_id,participant_status,order_id,experiment_key,driver_latitude,driver_longitude,driver_gps_accuracy
202865,2015-05-10 06:52:34,912428060012,CREATED,121051078603,118450977169,-2.972728,104.769280,4.291
202864,2015-05-10 06:52:49,912428060012,IGNORED,121051078603,118450977169,-2.972728,104.769280,4.291
97576,2015-05-10 06:52:49,266845890674,CREATED,121051078603,118450977169,-2.979929,104.773959,24.000
194448,2015-05-10 06:53:04,140511414730,CREATED,121051078603,118450977169,-2.976047,104.770823,27.508
97575,2015-05-10 06:53:04,266845890674,IGNORED,121051078603,118450977169,-2.979929,104.773959,24.000
194447,2015-05-10 06:53:05,140511414730,ACCEPTED,121051078603,118450977169,-2.976047,104.770823,27.508
36054,2015-05-10 06:54:51,737997071598,CREATED,121051078603,560933947856,-2.968593,104.761345,3.000
36065,2015-05-10 06:54:52,737997071598,ACCEPTED,121051078603,560933947856,-2.968593,104.761345,3.000
317408,2015-05-10 07:02:05,834534674631,CREATED,121051078603,355122673887,-2.972483,104.768002,27.314
317407,2015-05-10 07:02:20,834534674631,IGNORED,121051078603,355122673887,-2.972483,104.768002,27.314


In [64]:
df_booking_clean2.query('order_id == 121051078603').sort_values(['event_timestamp'], ascending=[True])

,event_timestamp,order_id,booking_status,customer_id,driver_id,trip_distance,pickup_latitude,pickup_longitude
610875,2015-05-10 06:52:33,121051078603,CREATED,215158762395,0,13.559,-2.97285,104.769132
610876,2015-05-10 06:53:06,121051078603,DRIVER_FOUND,215158762395,140511414730,13.559,-2.97285,104.769132


In [65]:
df_summary_participant = df_participant.merge(df_experiment, left_on=['order_id','experiment_key'], right_on=['order_id','experiment_key'])

In [66]:
df_summary_participant.head(5)

,event_timestamp_x,driver_id,participant_status,order_id,experiment_key,driver_latitude,driver_longitude,driver_gps_accuracy,event_timestamp_y,customer_id,experiment_tag
0,2015-05-18 11:45:19,110283102582,CREATED,938125906616,309579604593,-2.981389,104.761898,2.000,2015-05-18 11:45:04,786955815516,B
1,2015-05-18 11:45:29,110283102582,ACCEPTED,938125906616,309579604593,-2.981389,104.761898,2.000,2015-05-18 11:45:04,786955815516,B
2,2015-05-18 11:45:04,138719907917,CREATED,938125906616,309579604593,-2.979895,104.760992,5.436,2015-05-18 11:45:04,786955815516,B
3,2015-05-18 11:45:19,138719907917,IGNORED,938125906616,309579604593,-2.979895,104.760992,5.436,2015-05-18 11:45:04,786955815516,B
4,2015-05-11 10:35:57,126346998414,CREATED,992080359113,941995970460,-2.966915,104.748040,2.000,2015-05-11 10:35:56,758692050609,A


In [67]:
ap = df_summary_participant.query('participant_status == "IGNORED" or participant_status == "REJECTED"').groupby(['experiment_tag']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[True])

In [68]:
ap

,experiment_tag,size
0,A,2414
1,B,15750


In [69]:
bp = df_summary_participant.query('participant_status == "CREATED"').groupby(['experiment_tag']).size().to_frame('size').reset_index().sort_values(['size'], ascending=[True])

In [70]:
bp

,experiment_tag,size
0,A,20311
1,B,176588


In [71]:
print("results for A") 
M1_A = round(ap.iloc[0]['size']/bp.iloc[0]['size'], 2)
print(M1_A)

results for A
0.12


In [72]:
print("results for B") 
M1_B = round(ap.iloc[1]['size']/bp.iloc[1]['size'], 2)
print(M1_B)

results for B
0.09


In [73]:
df_summary_pivot['total_time'] = df_summary_pivot.apply(lambda row: row.COMPLETED - row.CREATED, axis=1)
df_summary_pivot['total_time_seconds'] = df_summary_pivot.apply(lambda row: (row.COMPLETED - row.CREATED).total_seconds(), axis=1)
df_summary_pivot['pick_up_time_seconds'] = df_summary_pivot.apply(lambda row: (row.PICKED_UP - row.CREATED).total_seconds(), axis=1)

In [74]:
df_summary_pivot.head(5)

,booking_status,COMPLETED,CREATED,CUSTOMER_CANCELLED,DRIVER_CANCELLED,DRIVER_FOUND,DRIVER_NOT_FOUND,PICKED_UP,pick_up_time,total_time,total_time_seconds,pick_up_time_seconds
order_id,experiment_tag,,,,,,,,,,,
100004630900,A,2015-05-12 00:27:09,2015-05-12 00:21:29,NaT,NaT,2015-05-12 00:21:30,NaT,2015-05-12 00:23:40,00:02:11,00:05:40,340.0,131.0
100005695817,A,2015-05-16 02:21:07,2015-05-16 02:09:16,NaT,NaT,2015-05-16 02:09:17,NaT,2015-05-16 02:12:06,00:02:50,00:11:51,711.0,170.0
100021292050,B,2015-05-10 13:34:21,2015-05-10 13:26:24,NaT,NaT,2015-05-10 13:26:26,NaT,2015-05-10 13:28:24,00:02:00,00:07:57,477.0,120.0
100022287505,B,2015-05-12 10:07:06,2015-05-12 09:50:51,NaT,NaT,2015-05-12 09:50:56,NaT,2015-05-12 09:54:29,00:03:38,00:16:15,975.0,218.0
100024200607,B,2015-05-11 04:57:03,2015-05-11 04:50:26,NaT,NaT,2015-05-11 04:50:27,NaT,2015-05-11 04:53:18,00:02:52,00:06:37,397.0,172.0


look at the data, see if data is skewed, or there are many fliers. If so, may want to use median instead

In [75]:
hist = df_summary_pivot.hist(bins=10, column='pick_up_time_seconds')
hist = df_summary_pivot.hist(bins=10, column='total_time_seconds')

look at the raw data for flyers, etc

In [76]:
df_summary_pivot.query('pick_up_time_seconds > 5000').head(5)

,booking_status,COMPLETED,CREATED,CUSTOMER_CANCELLED,DRIVER_CANCELLED,DRIVER_FOUND,DRIVER_NOT_FOUND,PICKED_UP,pick_up_time,total_time,total_time_seconds,pick_up_time_seconds
order_id,experiment_tag,,,,,,,,,,,
141035217409,B,2015-05-12 04:31:19,2015-05-12 02:53:49,NaT,NaT,2015-05-12 02:53:54,NaT,2015-05-12 04:23:10,01:29:21,01:37:30,5850.0,5361.0
150156182105,B,2015-05-12 04:52:48,2015-05-12 02:48:02,NaT,NaT,2015-05-12 02:48:04,NaT,2015-05-12 04:48:36,02:00:34,02:04:46,7486.0,7234.0
195170556711,B,2015-05-16 09:10:57,2015-05-16 05:50:57,NaT,NaT,2015-05-16 05:50:59,NaT,2015-05-16 09:10:51,03:19:54,03:20:00,12000.0,11994.0
329718614502,B,2015-05-10 13:27:29,2015-05-10 11:46:09,NaT,NaT,2015-05-10 11:46:10,NaT,2015-05-10 13:21:10,01:35:01,01:41:20,6080.0,5701.0
432291536014,B,2015-05-12 05:21:37,2015-05-12 02:44:01,NaT,NaT,2015-05-12 02:44:03,NaT,2015-05-12 05:14:43,02:30:42,02:37:36,9456.0,9042.0


looks like the long wait times are legitimate, they have a completed time that has a greater value also

In [77]:
at = df_summary_pivot.query('experiment_tag == "A"').agg({'total_time': ['mean']})

In [78]:
bt = df_summary_pivot.query('experiment_tag == "B"').agg({'total_time': ['mean']})

In [79]:
print("results for A") 
M2_A = round( at.iloc[0]['total_time'].total_seconds() / 60, 2)
print(round( at.iloc[0]['total_time'].total_seconds() / 60, 2))

results for A
15.43


In [80]:
print("results for B") 
M2_B = round( bt.iloc[0]['total_time'].total_seconds() / 60, 2)
print(round( bt.iloc[0]['total_time'].total_seconds() / 60, 2))

results for B
16.13


In [81]:
file = open("results.csv","w") 
 
file.write("Experiment Tag,Booking Conversion Rate,Mean Pick Up Time In Minutes,Another Metric,One More Metric" + "\n") 
file.write("A" + "," + str(BCR_A) + "," + str(MPUT_A) + "," + str(M1_A) + "," + str(M2_A) + "\n") 
file.write("B" + "," + str(BCR_B) + "," + str(MPUT_B) + "," + str(M1_B) + "," + str(M2_B) + "\n") 
 
file.close() 

In [82]:
results = pd.read_csv("results.csv")
results

,Experiment Tag,Booking Conversion Rate,Mean Pick Up Time In Minutes,Another Metric,One More Metric
0,A,0.75,4.76,0.12,15.43
1,B,0.83,4.81,0.09,16.13


write the clean dataframes to files to re-use in next assignment

In [83]:
df_booking_clean2.to_csv('booking_log_clean.csv', index=False, header=True)
df_participant.to_csv('participant_log_clean.csv', index=False, header=True)
df_experiment.to_csv('experiment_log_clean.csv', index=False, header=True)

The 1st "Another Metric" calculates the rate that the chosen driver will ignore or reject the order. We want to keep this to a minimum as it slows down the entire process. The 2nd "One More Metric" is the time from order creation to order completion, or passenger drop off. We also want this time minimized, as a shorter time will usually mean a happier customer.

Summary: 
It appears that method B (ML algorithm) is better in 2 success metrics, and worse in 2 other metrics, i.e. amount of time to move through the flow. This makes sense as method A is purely based on route, so the distance and therefore the times would be minimized. However, the closest driver may not be the best one to choose for a given order. There may be other variables that if included in the selection process, may help to improve driver acceptance and customer completion rates. Method B is superior in completion rate and in non-cancelled or ignored rates, indicating that the ML algorithm is better at picking a driver that is more likely to accept the order. In turn this will speed up the driver accepted time and give the customer the impression that there is a driver on the way, making him less likely to cancel the order. 